In [0]:
df1=spark.read.csv("/FileStore/tables/testfile.txt")
df1.show(truncate=False)

+---------+
|_c0      |
+---------+
|Name~|Age|
|umesh    |
|shruthi  |
|Shavvik  |
+---------+



In [0]:
df_filter=df1.rdd
rdd_filter=df_filter.zipWithIndex().filter(lambda x: x[1]>=1).map(lambda x: x[0])
df3=rdd_filter.toDF()
df3.show()

+-------+
|    _c0|
+-------+
|  umesh|
|shruthi|
|Shavvik|
+-------+



In [0]:
df=spark.read.csv("/FileStore/tables/azure_data_engineering_questions-1.txt")
df.show()

+--------------------+
|                 _c0|
+--------------------+
|pyspark----------...|
|1.repartition s c...|
|  2.cache vs persist|
|3.broadcast varia...|
|4.adaptive query ...|
| 5.shuffle partition|
|       6.partitionby|
|        7.checkpoint|
|8.data skew optim...|
|    9.job stage task|
|       10.parralizim|
|       11.serialize |
|      12.deserialize|
|13.salting   ----...|
|        14.bucketing|
|        15.skew hint|
|16.catalyst optim...|
|17.predicate push...|
| 18.RDD vs DATAframe|
|       19.RBO vs CBO|
+--------------------+
only showing top 20 rows



In [0]:
rdd=df.rdd
rdd_filtered=rdd.zipWithIndex().filter(lambda x: x[1]>5).map(lambda x : x[0])
df_rdd=rdd_filtered.toDF()
df_rdd.show(truncate=False)

+-----------------------------------------+
|_c0                                      |
+-----------------------------------------+
|6.partitionby                            |
|7.checkpoint                             |
|8.data skew optimization                 |
|9.job stage task                         |
|10.parralizim                            |
|11.serialize                             |
|12.deserialize                           |
|13.salting   ---- pseudo id              |
|14.bucketing                             |
|15.skew hint                             |
|16.catalyst optimizer                    |
|17.predicate pushdown projection pushdown|
|18.RDD vs DATAframe                      |
|19.RBO vs CBO                            |
|20. different file formats in spark      |
|21. snappy vs gzip                       |
|22.optimize command                      |
|23.spark streaming                       |
|24.dymanmic pruning                      |
+-------------------------------

In [0]:
#scenario 2 with headers
dbutils.fs.put("/FileStore/tables/empty_header.csv","""sampleline
smapleline2
sampleline3
id,name,location
1,ravi,bangalore
2,raj,chennai
3,prasad,pune
4,mahesh,hyderabad
5,sridhar,mumbai
""",True)

Wrote 133 bytes.
Out[1]: True

In [0]:
#since it is having variable columns 
df1_headers=spark.read.csv("/FileStore/tables/empty_header.csv")
df1_headers.show(truncate=False)

+-----------+
|_c0        |
+-----------+
|sampleline |
|smapleline2|
|sampleline3|
|id         |
|1          |
|2          |
|3          |
|4          |
|5          |
+-----------+



In [0]:
df_text=spark.read.text("/FileStore/tables/empty_header.csv")
df_text.show()

+------------------+
|             value|
+------------------+
|        sampleline|
|       smapleline2|
|       sampleline3|
|  id,name,location|
|  1,ravi,bangalore|
|     2,raj,chennai|
|     3,prasad,pune|
|4,mahesh,hyderabad|
|  5,sridhar,mumbai|
+------------------+



In [0]:
from pyspark.sql.functions import *
df_split=df_text.withColumn("split_column",split(col('value'),','))
df_split.show(truncate=0)


+------------------+----------------------+
|value             |split_column          |
+------------------+----------------------+
|sampleline        |[sampleline]          |
|smapleline2       |[smapleline2]         |
|sampleline3       |[sampleline3]         |
|id,name,location  |[id, name, location]  |
|1,ravi,bangalore  |[1, ravi, bangalore]  |
|2,raj,chennai     |[2, raj, chennai]     |
|3,prasad,pune     |[3, prasad, pune]     |
|4,mahesh,hyderabad|[4, mahesh, hyderabad]|
|5,sridhar,mumbai  |[5, sridhar, mumbai]  |
+------------------+----------------------+



In [0]:
df_map=df_text.rdd.zipWithIndex().filter(lambda x : x[1]>2)
df_map.collect()


Out[56]: [(Row(value='id,name,location'), 3),
 (Row(value='1,ravi,bangalore'), 4),
 (Row(value='2,raj,chennai'), 5),
 (Row(value='3,prasad,pune'), 6),
 (Row(value='4,mahesh,hyderabad'), 7),
 (Row(value='5,sridhar,mumbai'), 8)]

In [0]:
# The error AttributeError: split occurs because you are trying to call the split method on a PySpark Row object, but the Row object itself does not have a split method. Instead, you need to extract the value from the Row object first before calling split.
df_map=df_text.rdd.zipWithIndex().filter(lambda x : x[1]>2).map(lambda x: x[0].split(','))
df_map.collect()

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/databricks/python/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3378, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<command-2440898706884478>", line 3, in <module>
    df_map.collect()
  File "/databricks/spark/python/pyspark/instrumentation_utils.py", line 48, in wrapper
    res = func(*args, **kwargs)
  File "/databricks/spark/python/pyspark/rdd.py", line 1825, in collect
    sock_info = self.ctx._jvm.PythonRDD.collectAndServe(self._jrdd.rdd())
  File "/databricks/spark/python/lib/py4j-0.10.9.5-src.zip/py4j/java_gateway.py", line 1321, in __call__
    return_value = get_return_value(
  File "/databricks/spark/python/pyspark/errors/exceptions.py", line 228, in deco
    return f(*a, **kw)
  File "/databricks/spark/python/lib/py4j-0.10.9.5-src.zip/py4j/protocol.py", line 326, in get_return_value
    raise Py4JJavaError(
py4j.protocol.Py4JJavaError: An error occurred while calling z:org.apache.

In [0]:
df_map=df_text.rdd.zipWithIndex().filter(lambda x : x[1]>2).map(lambda x: x[0]['value'].split(','))
df_map.collect()

Out[69]: [['id', 'name', 'location'],
 ['1', 'ravi', 'bangalore'],
 ['2', 'raj', 'chennai'],
 ['3', 'prasad', 'pune'],
 ['4', 'mahesh', 'hyderabad'],
 ['5', 'sridhar', 'mumbai']]

In [0]:
# alternatively we can create rdd and split also 
rdd=sc.textFile("/FileStore/tables/empty_header.csv")
rdd.collect()

In [0]:
rdd_map=rdd.zipWithIndex().filter(lambda x : x[1]>2).map(lambda x: x[0].split(','))
rdd_map.first()

Out[65]: ['id', 'name', 'location']

In [0]:
columns=df_map.collect()[0]

In [0]:

df_final=df_map.filter(lambda x: x!=columns)
df_final.collect()

Out[90]: [['1', 'ravi', 'bangalore'],
 ['2', 'raj', 'chennai'],
 ['3', 'prasad', 'pune'],
 ['4', 'mahesh', 'hyderabad'],
 ['5', 'sridhar', 'mumbai']]

In [0]:
df_final.toDF(columns).show()

+---+-------+---------+
| id|   name| location|
+---+-------+---------+
|  1|   ravi|bangalore|
|  2|    raj|  chennai|
|  3| prasad|     pune|
|  4| mahesh|hyderabad|
|  5|sridhar|   mumbai|
+---+-------+---------+

